<a href="https://colab.research.google.com/github/CUSPADS2022IBX/IBXRidership/blob/main/Models/IBXRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Random Forest

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn                   #for plotting graphs
import plotly.graph_objects as go #for plotting graphs
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [3]:
#read valuyes 
mg_noIBX_abs = pd.read_csv('https://raw.githubusercontent.com/CUSPADS2022IBX/IBXRidership/main/mg_noIBX_abs.csv')
mg_noIBX_log = pd.read_csv('https://raw.githubusercontent.com/CUSPADS2022IBX/IBXRidership/main/mg_noIBX_log.csv')
mg_noIBX_norm = pd.read_csv('https://raw.githubusercontent.com/CUSPADS2022IBX/IBXRidership/main/mg_noIBX_norm.csv')

In [4]:
#fill NaNs in absolute column with 0s 
mg_noIBX_abs = mg_noIBX_abs.fillna(0)
mg_noIBX_norm = mg_noIBX_norm.fillna(0)
mg_noIBX_log = mg_noIBX_log.fillna(0)
#np.nan_to_num
mg_noIBX_log = mg_noIBX_log.replace([np.inf, -np.inf], 0)


In [5]:
#on log values 
pd.set_option("max_columns",None)
mg_noIBX_abs.head(3)
#weekday morning and weekday evening only peak ones 

,Unnamed: 0,complex_id,Stop Name,Complex_id,exits_weekday_evening,entries_weekday_evening,exits_weekday_morning,entries_weekday_morning,exits_weekday_overnight,entries_weekday_overnight,exits_weekend_evening,entries_weekend_evening,exits_weekend_morning,entries_weekend_morning,exits_weekend_overnight,entries_weekend_overnight,cs_ibx_peak,cs_ibx_offpeak,cs_noibx_peak,cs_noibx_offpeak,VoroID,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,otherarea,numbldgs,numfloors,unitsres,unitstotal,assesstot,Total_pop_Num,housed_pop_Num,group_house_pop_num,industrial_pop_num,under18_num,hispanic_num,white_num,black_num,asian_num,other_num,multirace_num,total_houses,occupied_homes_num,vacant_homes_num,Bus_Stops,geometry,Complex ID,boro,lat,long
0,0,614,59 ST-COLUMBUS CIRCLE,614c,10926.175,15541.200,25226.700,12127.425,892.400,679.500,9207.0625,8656.6875,13438.0000,6768.75,992.0000,907.3750,0.022481,0.013806,0.022649,0.013907,303.0,3.506512e+06,1.898716e+07,8.076521e+06,9.733897e+06,3.300706e+06,1.119935e+06,650293.456884,83286.493966,0.000000,2.922301e+06,183.121213,1747.992908,9694.778308,10595.702631,3.197204e+09,13952.842190,12686.163853,1266.678338,0.000000,1269.352811,1574.410869,8515.255032,616.540938,2565.146273,132.653509,548.835569,9177.614626,7595.313029,1582.301598,29.0,POLYGON ((990236.0543776825 218861.07537803546...,614,M,40.768271,-73.981832
1,1,611,42 ST-PORT AUTHORITY BUS TERMINAL,611c,47538.800,32175.925,51746.825,35712.750,3820.075,4421.525,36982.0625,23756.8125,32989.3125,18093.75,4894.6875,5036.8750,0.025420,0.016341,0.025604,0.016466,289.0,2.880067e+06,3.515431e+07,2.677124e+07,3.116553e+06,2.179797e+07,2.655222e+06,811092.979196,192855.064166,140027.608586,1.174067e+06,369.896651,3352.619951,4785.352598,9932.297027,5.270392e+09,6860.419037,6150.844036,709.575001,14.690317,456.485520,1227.369128,3214.088569,914.438730,1197.245697,70.786647,236.490267,4664.877276,4061.605283,603.271993,22.0,POLYGON ((986940.1379654831 213865.68954273767...,611,M,40.755746,-73.987581
2,2,601,14 ST,601c,8016.600,8628.425,13483.025,7083.525,697.075,459.025,6736.9375,5749.5000,8523.2500,3904.25,1071.5000,849.1875,0.024272,0.014947,0.024396,0.015019,338.0,2.164823e+06,1.237371e+07,4.752918e+06,7.347938e+06,2.607690e+06,1.193587e+06,146637.180360,33691.369486,946.797472,7.703662e+05,554.910817,2653.275670,7799.514511,8904.915810,1.820699e+09,11162.853868,10786.224803,376.629065,0.000000,1029.693323,900.853379,8471.750608,270.358591,928.680120,110.324878,480.886292,7395.664853,6489.408653,906.256200,11.0,"POLYGON ((984311.361812786 208635.71960600044,...",601,M,40.737796,-73.997732


In [6]:
#setting current so can re run everything easily (raw, absolute, log)
current = mg_noIBX_log

In [7]:

cols_housing = ['unitsres','unitstotal','total_houses', 'occupied_homes_num',
       'vacant_homes_num','resarea']


cols_dems = ['Total_pop_Num',
       'housed_pop_Num', 'group_house_pop_num', 'industrial_pop_num',
       'under18_num', 'hispanic_num', 'white_num', 'black_num', 'asian_num',
       'other_num', 'multirace_num']

In [8]:
from sklearn.preprocessing import StandardScaler

#composite score for housing 
x = StandardScaler().fit_transform(current[cols_housing])


from sklearn.decomposition import PCA
pca = PCA(n_components=1)
housing_composite = pca.fit_transform(x)


#and demographics 
x = StandardScaler().fit_transform(current[cols_dems])


from sklearn.decomposition import PCA
pca = PCA(n_components=1)
demo_composite = pca.fit_transform(x)
             




In [9]:
cols_short = ['lotarea', 'bldgarea', 'comarea', 'officearea', 'retailarea',
       'garagearea', 'strgearea', 'factryarea', 'otherarea', 'numbldgs',
       'numfloors', 'assesstot', 'Bus_Stops']

In [16]:
#cols for features except for centrality score --> fitting all in 
cols = ['lotarea', 'bldgarea', 'comarea', 'resarea', 'officearea', 'retailarea',
       'garagearea', 'strgearea', 'factryarea', 'otherarea', 'numbldgs',
       'numfloors', 'unitsres', 'unitstotal', 'assesstot', 'Total_pop_Num',
       'housed_pop_Num', 'group_house_pop_num', 'industrial_pop_num',
       'under18_num', 'hispanic_num', 'white_num', 'black_num', 'asian_num',
       'other_num', 'multirace_num', 'total_houses', 'occupied_homes_num',
       'vacant_homes_num', 'Bus_Stops']

In [17]:
#diff feature columns time depending 
cols_peak = cols + ['cs_noibx_peak']
cols_offpeak = cols + ['cs_noibx_offpeak']

In [18]:
#for use with PCA - only adding non PCA values, column selection
cols_peak = cols_short + ['cs_noibx_peak'] + ['housing_index'] + ['pop_index']
cols_offpeak = cols_short + ['cs_noibx_offpeak'] + ['housing_index'] + ['pop_index']

In [19]:
#merge with current
current = current.assign(housing_index=housing_composite)
current = current.assign(pop_index=demo_composite)

In [22]:
current.shape

(424, 58)

In [164]:
#target variables for ease of calling later 
#weekday
#scores for traditional random forest w/ 5 cv fold - R2 values 
#first score is absolute value 
#try PCA 
exits_weekday_morning = current['exits_weekday_morning'] #.171
entries_weekday_morning = current['entries_weekday_morning'] #-.16

exits_weekday_evening = current['exits_weekday_evening'] #-.20 
entries_weekday_evening = current['entries_weekday_evening'] #.046

exits_weekday_overnight = current['exits_weekday_overnight'] #-.15
entries_weekday_overnight = current['entries_weekday_overnight'] #-.12

#weekend
exits_weekend_morning = current['exits_weekend_morning'] #0.04
entries_weekend_morning = current['entries_weekend_morning'] #-0.097

exits_weekend_evening = current['exits_weekend_evening'] #.01
entries_weekend_evening = current['entries_weekend_evening'] #.127

exits_weekend_overnight = current['exits_weekend_overnight'] #-.24
entries_weekend_overnight = current['entries_weekend_overnight'] #0.04 


In [165]:
#list of target values 
trip_targets = [exits_weekday_morning,entries_weekday_morning,exits_weekday_evening,entries_weekday_evening,exits_weekday_overnight,entries_weekday_overnight,
                exits_weekend_morning,entries_weekend_morning,exits_weekend_evening,entries_weekend_evening,exits_weekend_overnight,entries_weekend_overnight]

In [ ]:
#

In [166]:
#setting peak values 
peak = [exits_weekday_morning,entries_weekday_morning,exits_weekday_evening,entries_weekday_evening]

#looping through 
for target in trip_targets:
    from sklearn.model_selection import cross_val_score

    rf = RandomForestRegressor(random_state = 42,n_jobs=-1,n_estimators=50)

    if target.name in map(lambda x: x.name,peak):
      scores = cross_val_score(rf, current[cols_peak], target, cv=5)
    else:
      scores = cross_val_score(rf, current[cols_offpeak], target, cv=5)

    print(target.name, 'R2Score:' ,scores.mean())


    #lets tune --> use grid search cv to tune 
    #later use this to fit - play with parameters 
    #cross validation
    #grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

    #rf.fit(current[cols_offpeak],exits_weekday_morning)

exits_weekday_morning R2Score: 0.38700315586132594
entries_weekday_morning R2Score: 0.17424130394104503
exits_weekday_evening R2Score: 0.11332539455662574
entries_weekday_evening R2Score: 0.39259888222436845
exits_weekday_overnight R2Score: 0.024850459931700385
entries_weekday_overnight R2Score: 0.1484572469266146
exits_weekend_morning R2Score: 0.3159314924857951
entries_weekend_morning R2Score: 0.11775391716142095
exits_weekend_evening R2Score: 0.204465911207937
entries_weekend_evening R2Score: 0.33473422634231076
exits_weekend_overnight R2Score: 0.00021854928146836095
entries_weekend_overnight R2Score: 0.2534040053249148


In [167]:
rf.fit(current[cols_offpeak],entries_weekend_evening)
feature_scores = rf.feature_importances_

In [168]:
test = dict(zip(cols_peak,feature_scores))


sorted(test.items(), key=lambda item: item[1])

#use PCA to get composite score for housing and demographics 
#mixness score 

[('numfloors', 0.01062016847384322),
 ('pop_index', 0.01458061321138317),
 ('strgearea', 0.017659058060729677),
 ('otherarea', 0.020339994630067363),
 ('factryarea', 0.021259755463374875),
 ('comarea', 0.02167035947212359),
 ('garagearea', 0.023389737643148764),
 ('bldgarea', 0.025241690195608693),
 ('housing_index', 0.027361673175454265),
 ('lotarea', 0.02935711980518353),
 ('Bus_Stops', 0.037058899208302035),
 ('officearea', 0.04600819921827503),
 ('numbldgs', 0.05245743429221167),
 ('retailarea', 0.1050013056973529),
 ('cs_noibx_peak', 0.20940509566768103),
 ('assesstot', 0.3385888957852602)]

In [125]:
#test = pd.concat(feature_scores,cols_peak)
feature_scores

array([0.01666741, 0.03110036, 0.03222529, 0.00316789, 0.07408091,
       0.04584445, 0.01360969, 0.01188481, 0.00740225, 0.0176281 ,
       0.02981948, 0.00748431, 0.00278372, 0.00333722, 0.19584361,
       0.00327011, 0.00218424, 0.0100469 , 0.00932198, 0.00729952,
       0.00938245, 0.00684932, 0.00741319, 0.01338993, 0.00974629,
       0.00744318, 0.0020968 , 0.00219944, 0.01552973, 0.02615048,
       0.37479692])

In [126]:
cols_peak

['lotarea',
 'bldgarea',
 'comarea',
 'resarea',
 'officearea',
 'retailarea',
 'garagearea',
 'strgearea',
 'factryarea',
 'otherarea',
 'numbldgs',
 'numfloors',
 'unitsres',
 'unitstotal',
 'assesstot',
 'Total_pop_Num',
 'housed_pop_Num',
 'group_house_pop_num',
 'industrial_pop_num',
 'under18_num',
 'hispanic_num',
 'white_num',
 'black_num',
 'asian_num',
 'other_num',
 'multirace_num',
 'total_houses',
 'occupied_homes_num',
 'vacant_homes_num',
 'Bus_Stops',
 'cs_noibx_peak']

In [ ]:
#cross validation - basically doing train + test comparison to get best trained model 
#get crossvalscore --> 

In [ ]:
#how to validate predictions

In [ ]:
#make chloropleth with centrality by voronoi with and without IBX --> map before and after 
#check if polygons still in final table - output and merge with scores - color accordingly 

In [ ]:
#use Turbold model to predict entries and exits across all time periods for IBX dataset 
#use ins and outs to build gravity model 